## Load the SVElearnj package

In [ ]:
using Pkg
Pkg.add(url="https://github.com/giordamaug/SVElearnj")

## SVE of RandomForests on Irisi dataset

In [ ]:
using MLJ
using MLJBase
using RDatasets
using SVElearnj: SplitVotingEnsemble

# Load the Iris dataset
iris = dataset("datasets", "iris")
X = Matrix(iris[:, 1:4])  # Manually select feature columns
X = DataFrame(X, :auto)
y = iris[:, 5] 

# Partition the dataset
train, test = MLJBase.partition(eachindex(y), 0.8, shuffle=true, rng=1)

# Load and initialize the classifier
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree verbosity=-1
clf = DecisionTreeClassifier(max_depth=5)
sve_model = SplitVotingEnsemble(clf)

# Bind model to data with a machine
mach = machine(sve_model, X, y)
fit!(mach)

# Make predictions
results = evaluate!(
    mach,
    check_measure=false,
    resampling=MLJ.CV(nfolds=5),
    measure = [accuracy, sensitivity, specificity, balanced_accuracy, mcc, confmat],
    per_observation=false,
    verbosity=0
)
conf_matrix = results.measurement[6]